In [179]:
# Для преобразования типа данных столбцов в test_v2.csv
# dtypes_list = x.dtypes
# dtypes_list = list(dtypes_list)
# dtypes = open('data/dtypes.txt', 'w')
# for item in dtypes_list:
#     dtypes.write("%s\n" % item)
# dtypes.close()

# Для чтения типов данных столбцов из train_v2.csv
# dtypes = open('data/dtypes.txt', 'r')
# ls = []
# for item in dtypes:
#     ls.append(item.rstrip())

from IPython.display import display
import numpy as np
import pandas as pd
import catboost
import json
import time
import os
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score


#Внимание!!! БЫДЛОКОД!
def calc_level(hero_xp):
    if math.isnan(hero_xp): return 0
    elif int(hero_xp) < 200: return 1
    elif int(hero_xp) < 500: return 2
    elif int(hero_xp) < 900: return 3
    elif int(hero_xp) < 1400: return 4
    elif int(hero_xp) < 2000: return 5
    elif int(hero_xp) < 2615: return 6
    elif int(hero_xp) < 3425: return 7
    elif int(hero_xp) < 3890: return 8
    elif int(hero_xp) < 4550: return 9
    elif int(hero_xp) < 5225: return 10
    elif int(hero_xp) < 6000: return 11
    elif int(hero_xp) < 7175: return 12
    elif int(hero_xp) < 8375: return 13
    elif int(hero_xp) < 9600: return 14
    elif int(hero_xp) < 10850: return 15
    elif int(hero_xp) < 12125: return 16
    elif int(hero_xp) < 13500: return 17
    elif int(hero_xp) < 14900: return 18
    elif int(hero_xp) < 16325: return 19
    elif int(hero_xp) < 17925: return 20
    elif int(hero_xp) < 19825: return 21
    elif int(hero_xp) < 22025: return 22
    elif int(hero_xp) < 24525: return 23
    elif int(hero_xp) < 27500: return 24
    else: return 25

def add_lvl_column(table, xp_col_list, lvl_col_list):
    for xp_col, lvl_col in zip(xp_col_list,lvl_col_list):
        table[lvl_col] = table[xp_col].apply(calc_level)
#     for lvl_col in lvl_col_list:
#         table[lvl_col] = pd.to_numeric(table[lvl_col])
    return table

# Формула: (int(str_gain*(lvl - 1)) + base_str)*20 + base_health
def get_current_health(data):
    return ((int(data[3]*(data[0] - 1)) + data[2])*20 + data[1])

# Формула: (int(agi_gain*(lvl - 1)) + base_agi)*(1/6) + base_armor
def get_current_armor(data):
    return round(((int(data[3]*(data[0] - 1)) + data[2])*(1/6) + data[1]), 2)

# Формула: (int(int_gain*(lvl - 1)) + base_int)*12 + base_mana
def get_current_mana(data):
    return ((int(data[3]*(data[0] - 1)) + data[2])*12 + data[1])

def add_properties_columns(table, heroes, player_list, lvl_list, props_list):
    i = 1
    for player_col, lvl_col in zip(player_list, lvl_list):
        temp_table = pd.DataFrame(table[[player_col, lvl_col]])
        temp_table = temp_table.join(heroes[props_list], on=player_col)
        temp_table = temp_table.set_index(player_col)
        health_properties = props_list[0:3]
        armor_properties = props_list[3:6]
        mana_properties = props_list[6:9]
        temp_table = temp_table.fillna(0)
        temp_table[['base_health', 'base_str', 'base_armor', 'base_agi', 'base_mana', 'base_int']] = temp_table[['base_health', 'base_str', 'base_armor', 'base_agi', 'base_mana', 'base_int']].astype(int)
        health_properties.insert(0, lvl_col)
        armor_properties.insert(0, lvl_col)
        mana_properties.insert(0, lvl_col)
        props_list.insert(0, lvl_col)
#         temp_table[props_list] = temp_table[props_list]
#         index = list(temp_table[lvl_col].index[temp_table[lvl_col].apply(np.isnan)])
        temp_table['{}_current_health'.format(player_col)] = temp_table[health_properties].apply(get_current_health, axis=1)
        temp_table['{}_current_armor'.format(player_col)] = temp_table[armor_properties].apply(get_current_armor, axis=1)
        temp_table['{}_current_mana'.format(player_col)] = temp_table[mana_properties].apply(get_current_mana, axis=1)
        temp_table = temp_table.reset_index()
        temp_table.drop(player_col, axis=1, inplace=True)
        temp_table.drop(props_list, axis=1, inplace=True)
        del props_list[0]
        temp_table.columns = ['{}_current_health'.format(player_col), 
                              '{}_current_armor'.format(player_col), 
                              '{}_current_mana'.format(player_col)]
        table = table.reset_index()
        temp_table['fight_id'] = table['fight_id']
        temp_table = temp_table.set_index('fight_id')
        temp_table[['{}_current_health'.format(player_col), '{}_current_mana'.format(player_col)]] = temp_table[['{}_current_health'.format(player_col), '{}_current_mana'.format(player_col)]].astype(int)
        table = table.join(temp_table[[
            '{}_current_health'.format(player_col),
            '{}_current_armor'.format(player_col),
            '{}_current_mana'.format(player_col)
        ]], on='fight_id')
        table = table.set_index('fight_id')
        print(i, " step: Completed")
        i += 1
        
    table['health_diff'] = (table['r1_hero_current_health'] + table['r2_hero_current_health'] + table['r3_hero_current_health'] + table['r4_hero_current_health'] + table['r5_hero_current_health']) - (table['d1_hero_current_health'] + table['d2_hero_current_health'] + table['d3_hero_current_health'] + table['d4_hero_current_health'] + table['d5_hero_current_health'])
    table['armor_diff'] = (table['r1_hero_current_armor'] + table['r2_hero_current_armor'] + table['r3_hero_current_armor'] + table['r4_hero_current_armor'] + table['r5_hero_current_armor']) - (table['d1_hero_current_armor'] + table['d2_hero_current_armor'] + table['d3_hero_current_armor'] + table['d4_hero_current_armor'] + table['d5_hero_current_armor'])
    table['mana_diff'] = (table['r1_hero_current_mana'] + table['r2_hero_current_mana'] + table['r3_hero_current_mana'] + table['r4_hero_current_mana'] + table['r5_hero_current_mana']) - (table['d1_hero_current_mana'] + table['d2_hero_current_mana'] + table['d3_hero_current_mana'] + table['d4_hero_current_mana'] + table['d5_hero_current_mana'])
    
    category_for_drop = []
    drop_file = open('data/drop_2.txt')
    for line in drop_file:
        line = line.split('\n')
        category_for_drop.append(line[0])
    table.drop(category_for_drop, axis=1, inplace=True)
    
    return table

def has_group_heroes(data, firstId, secondId):
    hasFirst = False
    hasSecond = False
    for hero in data:
        if int(hero) == int(firstId):
            hasFirst = True
            continue
        if int(hero) == int(secondId):
            hasSecond = True
            continue
    if hasFirst and hasSecond:
        return True
    else: return False    

def add_group_columns(table, groups_list, firsId_list, secondId_list, teams_list, player_list):
#     temp_table = table.loc[:, player_list]
    player = player_list[0:5]
    for team in teams:
        for group, firstId, secondId in zip(groups_list, firsId_list, secondId_list):
            table['{0}{1}'.format(team, group)] = table[player].apply(has_group_heroes, firstId=firstId, secondId=secondId, axis=1)
        player = player_list[5:]
    return table

player_list = ['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
               'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']

player_xp_list = ['r1_xp','r2_xp','r3_xp','r4_xp','r5_xp',
                  'd1_xp','d2_xp','d3_xp','d4_xp','d5_xp']

lvl_list = ['r1_lvl','r2_lvl','r3_lvl','r4_lvl','r5_lvl',
                  'd1_lvl','d2_lvl','d3_lvl','d4_lvl','d5_lvl']

properties_list = [
    'base_health', 'base_str', 'str_gain',
    'base_armor', 'base_agi', 'agi_gain',
    'base_mana', 'base_int', 'int_gain'
]

groups_list = [
    'Juggernaut + Crystal maiden', 'Bane + Mirana', 'Shadow shaman + Ursa', 'Dazzle + Huskar',
    'Magnus + Dark seer', 'Pudge + Rubick', 'Lion + Lina', 'Tiny + Io',
    'Bloodseeker + Invoker', 'Lifestealer + Spirit breaker', 'Lifestealer + Storm spirit ', 'Lifestealer + Nyx assassin',
    'Faceless Void + Witch doctor', 'Chaos knight + Io', 'Faceless Void + Lich'
]

firsId_list = [
    8, 3, 27, 50, 97, 14, 26, 19, 4, 54, 54, 54, 41, 81, 41
]

secondId_list = [
    5, 9, 70, 59, 55, 86, 25, 91, 74, 71, 17, 88, 30, 91, 31
]

teams_list = ['(radiant) ', '(dire) ']


x = pd.read_csv('data/train_v2.csv', index_col=0)
y = pd.read_csv('data/target.csv', index_col=0)['radiant_won']

# Установили значение вывода DataFrame таблиц во всю ширину
pd.options.display.max_columns = 379

# Удаляем все колонки, практически не влияющие на исход драки
category_for_drop = []
drop_file = open('data/drop.txt')
for line in drop_file:
    line = line.split('\n')
    category_for_drop.append(line[0])
x.drop(category_for_drop, axis=1, inplace=True)

# Получаем DataFrame всех героев и их свойств
with open('data/heroes.json') as file:
    heroes = json.load(file)
df_heroes = {
    int(hero_id): hero_params
    for hero_id, hero_params in heroes.items()
}

df_heroes = pd.DataFrame(df_heroes).T
df_heroes = df_heroes.infer_objects()

index = list(x['r1_hero'].index[x['r1_hero'].apply(np.isnan)])
x.loc[index, list(x.columns)] = 0

dtypes = open('data/dtypes.txt', 'r')
for dtype, col in zip(dtypes, x):
    x[col] = x[col].astype(dtype.rstrip())

# Добавляем колонку уровней игрока
x = add_lvl_column(x, player_xp_list, lvl_list)

# Добавляем колонки жизней, ловкости и маны
# x = add_properties_columns(x, df_heroes, player_list, lvl_list, properties_list)

# path = 'DataFrames/x_for_train_v1.csv'
# if not os.path.exists('DataFrames'):
#     os.makedirs('DataFrames')
# x.to_csv(path, index=True)


x['kda_diff'] = (x['r1_avg_kda'] + x['r2_avg_kda'] + x['r3_avg_kda'] + x['r4_avg_kda'] + x['r5_avg_kda']) - (x['d1_avg_kda'] + x['d2_avg_kda'] + x['d3_avg_kda'] + x['d4_avg_kda'] + x['d5_avg_kda'])
x['kills_diff_before_fight'] = (x['r1_kills'] + x['r2_kills'] + x['r3_kills'] + x['r4_kills'] + x['r5_kills']) - (x['d1_kills'] + x['d2_kills'] + x['d3_kills'] + x['d4_kills'] + x['d5_kills'])
x['tower_damage_per_min_diff'] = (x['r1_avg_tower_damage_per_min'] + x['r2_avg_tower_damage_per_min'] + x['r3_avg_tower_damage_per_min'] + x['r4_avg_tower_damage_per_min'] + x['r5_avg_tower_damage_per_min']) - (x['d1_avg_tower_damage_per_min'] + x['d2_avg_tower_damage_per_min'] + x['d3_avg_tower_damage_per_min'] + x['d4_avg_tower_damage_per_min'] + x['d5_avg_tower_damage_per_min'])
x['gold_diff'] = (x['r1_gold'] + x['r2_gold'] + x['r3_gold'] + x['r4_gold'] + x['r5_gold']) - (x['d1_gold'] + x['d2_gold'] + x['d3_gold'] + x['d4_gold'] + x['d5_gold'])
x['deaths_diff'] = (x['r1_deaths'] + x['r2_deaths'] + x['r3_deaths'] + x['r4_deaths'] + x['r5_deaths']) - (x['d1_deaths'] + x['d2_deaths'] + x['d3_deaths'] + x['d4_deaths'] + x['d5_deaths'])
x['hl_diff'] = (x['r1_lh'] + x['r2_lh'] + x['r3_lh'] + x['r4_lh'] + x['r5_lh']) - (x['d1_lh'] + x['d2_lh'] + x['d3_lh'] + x['d4_lh'] + x['d5_lh'])

x['xp_diff'] = (x['r1_xp'] + x['r2_xp'] + x['r3_xp'] + x['r4_xp'] + x['r5_xp']) - (x['d1_xp'] + x['d2_xp'] + x['d3_xp'] + x['d4_xp'] + x['d5_xp'])
x['items_diff'] = (x['r1_items'] + x['r2_items'] + x['r3_items'] + x['r4_items'] + x['r5_items']) - (x['d1_items'] + x['d2_items'] + x['d3_items'] + x['d4_items'] + x['d5_items'])
# x['health_diff'] = (x['r1_hero_current_health'] + x['r2_hero_current_health'] + x['r3_hero_current_health'] + x['r4_hero_current_health'] + x['r5_hero_current_health']) - (x['d1_hero_current_health'] + x['d2_hero_current_health'] + x['d3_hero_current_health'] + x['d4_hero_current_health'] + x['d5_hero_current_health'])


# x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.33, random_state=1)

# scaler = StandardScaler()

# x_train = scaler.fit_transform(x_train)
# x_validation = scaler.fit_transform(x_validation)

# features = list(x.columns)

In [180]:
prefix_players = ['r1', 'r2', 'r3', 'r4', 'r5', 'd1', 'd2', 'd3', 'd4', 'd5']
columns = ['_avg_kda', '_kills', '_avg_tower_damage_per_min', '_gold', '_deaths', '_lh', '_xp', '_items']

category_for_drop = []
for prefix in prefix_players:
    for col in columns:
        category_for_drop.append('{0}{1}'.format(prefix, col))
x.drop(category_for_drop, axis=1, inplace=True)

In [210]:
with bz2.BZ2File('data/train.jsonlines.bz2') as f:
    for line in f:
        radiant_kills = 0
        dire_kills = 0
        info = json.loads(line)
        for item in info['events']:
            for prefix in prefix_players[:5]:
                if (item['type'] == '{}_kills_log'.format(prefix) and 
                    item['time'] >= x.loc[info['fight_id'], 'teamfight_start']):
                    radiant_kills += 1
            for prefix in prefix_players[5:]:
                if (item['type'] == '{}_kills_log'.format(prefix) and 
                    item['time'] >= x.loc[info['fight_id'], 'teamfight_start']):
                    dire_kills += 1
        data.append(radiant_kills - dire_kills)


[3,
 -1,
 -2,
 -1,
 -2,
 1,
 -1,
 -1,
 -1,
 2,
 -1,
 -2,
 1,
 1,
 0,
 1,
 2,
 -1,
 -3,
 2,
 -2,
 0,
 -2,
 -2,
 0,
 5,
 -3,
 1,
 0,
 1,
 0,
 1,
 1,
 -2,
 0,
 -2,
 -1,
 -2,
 -1,
 -1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 -2,
 -1,
 0,
 0,
 1,
 1,
 2,
 -2,
 2,
 -1,
 1,
 2,
 1,
 0,
 -3,
 -2,
 -1,
 -1,
 0,
 0,
 -2,
 1,
 0,
 2,
 0,
 1,
 2,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -2,
 1,
 1,
 0,
 -1,
 0,
 -2,
 0,
 -1,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 1,
 1,
 1,
 -2,
 1,
 -2,
 -1,
 0,
 0,
 -1,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 -1,
 -2,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 1,
 2,
 0,
 -1,
 -3,
 2,
 0,
 0,
 -1,
 -2,
 1,
 -1,
 0,
 1,
 -1,
 1,
 2,
 -1,
 -4,
 0,
 1,
 -1,
 0,
 -1,
 -2,
 -1,
 0,
 3,
 -2,
 1,
 2,
 1,
 -2,
 0,
 2,
 -1,
 0,
 1,
 0,
 -1,
 0,
 1,
 1,
 -2,
 3,
 -2,
 0,
 -1,
 0,
 -2,
 1,
 1,
 2,
 1,
 1,
 -2,
 -1,
 0,
 2,
 1,
 -2,
 0,
 1,
 2,
 -1,
 2,
 0,
 0,
 2,
 3,
 1,
 0,
 -2,
 1,
 -1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 -2,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 1,
 -1,
 -1,
 -1,
 0,
 -2,
 2,
 1,
 0,
 -1,
 0,
 1,
 -1,
 0,
 -1,


In [224]:
data = pd.DataFrame(data)
data.reset_index(inplace=True)

In [225]:
data.columns = ['fight_id', 'kills_diff_on_teamfight']

,index,0
0,0,3
1,1,-1
2,2,-2
3,3,-1
4,4,-2
5,5,1
6,6,-1
7,7,-1
8,8,-1
9,9,2
